In [1]:
import pandas as pd
import numpy
from pandas import DataFrame
import csv

In [2]:
# reading in the csv file
original = pd.read_csv('CleanedGunViolenceData.csv')

In [3]:
original.head()

,date,state,city_or_county,n_killed,n_injured,congressional_district,gun_type,incident_characteristics,latitude,location_description,...,female participants,male participants,arrested,unharmed,unharmed/arrested,victims,subject/suspect,children,teens,adults
0,1/1/2014,Kentucky,Lexington,0,1,6.0,NaN,Shot - Wounded/Injured,38.0507,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1/1/2014,Kentucky,Cynthiana,0,1,4.0,0::Unknown,Shot - Wounded/Injured||Accidental Shooting||A...,38.4333,NaN,...,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0
2,1/1/2014,Kentucky,Louisville,0,1,3.0,NaN,Shot - Wounded/Injured,38.2501,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1/1/2014,Virginia,Falls Church,0,0,8.0,NaN,Armed robbery with injury/death and/or evidenc...,38.8847,NaN,...,NaN,NaN,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN
4,1/1/2014,Oklahoma,Lawton,0,0,4.0,NaN,Shots Fired - No Injuries,34.6347,NaN,...,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0


In [5]:

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [6]:
%pyspark
# Sample DataFrame with repeating words
GVdataframe = spark.original['incident_characteristics']
GVdataframe.show()

UsageError: Line magic function `%pyspark` not found.


In [ ]:
%pyspark
# Tokenize the words
tokenizer = Tokenizer(inputCol="incident_characteristics", outputCol="tokens")
GVwordsData = tokenizer.transform(GVdataframe)
GVwordsData.show()

In [ ]:
%pyspark
# Run the hashing term frequency
hashing = HashingTF(inputCol="tokens", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(GVwordsData)

In [ ]:
%pyspark
# Display new DataFrame
hashed_df.show(truncate=False)

In [ ]:
%pyspark
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [ ]:
%pyspark
# Display the DataFrame
rescaledData.select("incident_characteristics", "features").show(truncate=False)